<h2>Final Exam</h2>
You are a stock analyst working in the research department of a small<br>
hedge fund.  Your job is to analyze the returns of a pool of securities<br>
and back test the quants strategy of buys and sells.  You need<br>
to load the 4 securities and test their strategy.<br>
files:<br>
INTC - Intel Corporation<br>
AMD - Advanced Micro Devices<br>
AAPL - Apple<br>
FB - Meta Platforms, Inc.<br>
corporations = {'INTC': 'Intel Corporation',<br>
'AMD':'Advanced Micro Devices',<br>
'AAPL':'Apple',<br>
'FB':'Meta Platforms, Inc.'}<br>
<br>
NOTE: All trades use the closing price!!!!

<h2>Step 1</h2>
<h4>5 points</h4>
Create a directory and place all of the csv files into it<br>
Create the variable proj_dir = '  <your directory> '<br>
db_name = 'stock_analysis'

In [422]:
import os 
import sqlite3
proj_dir = '/Users/julis_478/Documents/Python Immersives/PY-112 SQL, No SQL/Classes/Activities Github/Final_Test/Folder_2'
try:
    os.mkdir(proj_dir)
except:
    pass
os.chdir(f'{proj_dir}')
print (os.getcwd())

/Users/julis_478/Documents/Python Immersives/PY-112 SQL, No SQL/Classes/Activities Github/Final_Test/Folder_2


<h2>Step 2</h2>
<h4>5 points</h4>
Create a database called 'Stock_Analysis'

In [423]:
# Create your database here
db_name = 'stock_analysis'
conn = sqlite3.connect(f'{db_name}.db')

<h2>Step 3</h2>
<h4>15 points</h4>
Create the following tables:<br>
1 - security_master<br>
symbol<br>
trade_date<br>
open_price<br>
high_price<br>
low_price<br>
adj_close<br>
close_price<br>
volume<br>
<br>
2 - security_info<br>
symbol<br>
name  Max of 25 characters<br>
research_notes Max of 255 characters<br>
<br>
3 - security_log<br>
transaction_id  Character max 20 alpha numeric<br>
trade_date<br>
symbol<br>
trans_type   - (will contain only 'BUY' or 'SELL')<br>
shares   (Will only contain whole numbers)<br>
cost - Will hold dollar amounts<br>
balance - Will hold dollar amounts<br>

In [424]:
import sqlite3
import os

conn = sqlite3.connect(f'stock_analysis.db')
c = conn.cursor()

c.executescript('''
    create table IF NOT EXISTS security_master (
        symbol TEXT,
        trade_date TEXT,
        open_price FLOAT,
        high_price FLOAT,
        low_price FLOAT,
        adj_close FLOAT,
        close_price FLOAT,
        volume INTEGER
    );
    DELETE FROM security_master;

    create table IF NOT EXISTS security_info(
        symbol TEXT, 
        name TEXT,  
        research_notes TEXT
    );
    DELETE FROM security_info;

    create table IF NOT EXISTS security_log(
        transaction_id TEXT,
        trade_date TEXT,
        symbol TEXT,
        trans_type TEXT,
        shares INTEGER,  
        cost FLOAT,  
        balance FLOAT
    );
    DELETE FROM security_log;

    ''')

conn.commit()
conn.close()

<h2>Step 4</h2>
<h4>15 points</h4>
Load the security master table<br>
Your code should read whatever files that are csv files in the directory<br>
Use the dictionary below to populate the security_info table:<br>
corporations = {'INTC': 'Intel Corporation',
'AMD':'Advanced Micro Devices',
'AAPL':'Apple',
'FB':'Meta Platforms, Inc.'}

In [425]:
import glob
import os

conn = sqlite3.connect(f'stock_analysis.db')
c = conn.cursor()

corporations = {'INTC': 'Intel Corporation',
                'AMD':'Advanced Micro Devices',
                'AAPL':'Apple',
                'FB':'Meta Platforms, Inc.'}
c.execute('DELETE FROM security_master')
conn.commit()
for i,files in enumerate (glob.glob('*.csv')):
    with open(f'{files}', 'r') as f:
        headline = f.readline()
        lines = f.readlines()
        simbol = files.replace('.csv', '')
        print (simbol)
        for i in lines:
            date,open_p,high_p,low_p,close_p,adj_close,volume = i.replace('\n','').split(',')
            open_p,high_p,low_p,close_p,adj_close,volume = float(open_p),float(high_p),float(low_p),float(close_p),float(adj_close),int(volume)
            pop_data = (simbol, date,open_p,high_p,low_p,close_p,adj_close,volume)
            c.execute('''INSERT INTO security_master (symbol, trade_date, open_price, high_price, low_price, adj_close, close_price, volume) VALUES (?,?,?,?,?,?,?,?)''',pop_data)
    conn.commit()
conn.close()

INTC
AMD
FB
AAPL


In [426]:
import glob
import os

conn = sqlite3.connect(f'stock_analysis.db')
c = conn.cursor()

corporations = {'INTC': 'Intel Corporation',
                'AMD':'Advanced Micro Devices',
                'AAPL':'Apple',
                'FB':'Meta Platforms, Inc.'}

c.execute('DELETE FROM security_info')
conn.commit()
for symbols, names in corporations.items():
    c.execute(f"INSERT INTO security_info (symbol, name) VALUES ('{symbols}','{names}')")
conn.commit()
conn.close()


In [427]:
# (BORAR)

import glob
import os

conn = sqlite3.connect(f'stock_analysis.db')
c = conn.cursor()

c.execute('SELECT * FROM security_info ') 
into_data = c.fetchall()
print (into_data)
conn.close()

[('INTC', 'Intel Corporation', None), ('AMD', 'Advanced Micro Devices', None), ('AAPL', 'Apple', None), ('FB', 'Meta Platforms, Inc.', None)]


<h2>Step 5</h2>
<h4>10 points</h4>
Print the symbol,name, and number of records of every security using 1 SQL statement<br>

In [428]:
import glob
import os

conn = sqlite3.connect(f'stock_analysis.db')
c = conn.cursor()

corporations = {'INTC': 'Intel Corporation',
                'AMD':'Advanced Micro Devices',
                'AAPL':'Apple',
                'FB':'Meta Platforms, Inc.'}

c.execute('SELECT A.symbol, A.name, count(B.symbol) FROM security_info A LEFT JOIN security_master B ON A.symbol = B.symbol GROUP BY B.symbol') 
into_data = c.fetchall()
[print (f'{i[0]} - {i[1]}: \n Number of records: {i[2]}') for i in into_data]



AAPL - Apple: 
 Number of records: 754
AMD - Advanced Micro Devices: 
 Number of records: 754
FB - Meta Platforms, Inc.: 
 Number of records: 754
INTC - Intel Corporation: 
 Number of records: 754


[None, None, None, None]

<h2>Step 6</h2>
<h4>10 points</h4>
Print the average closing share price for each security in the year 2019<br>

In [429]:
import glob
import os

conn = sqlite3.connect(f'stock_analysis.db')
c = conn.cursor()

c.execute('SELECT symbol, AVG(close_price) FROM security_master WHERE trade_date LIKE "2019%" GROUP BY symbol') 
into_data = c.fetchall()
[print (f'The average closing price for {i[0]}: {round(i[1],2)}') for i in into_data]
conn.close()


The average closing price for AAPL: 51.0
The average closing price for AMD: 29.94
The average closing price for FB: 181.64
The average closing price for INTC: 48.32


<h2>Step 7</h2>
<h4>5 points</h4>
Print the max and min closing share price for each security in the year 2020<br>

In [430]:
import glob
import os

conn = sqlite3.connect(f'stock_analysis.db')
c = conn.cursor()

c.execute('SELECT symbol, MAX(close_price), MIN(close_price) FROM security_master WHERE trade_date LIKE "2020%" GROUP BY symbol') 
into_data = c.fetchall()
[print (f'The closing share price for {i[0]}: ({round(i[1],2)}, {round(i[2],2)})') for i in into_data]
conn.close()

The closing share price for AAPL: (135.85, 55.4)
The closing share price for AMD: (97.12, 38.71)
The closing share price for FB: (303.91, 146.01)
The closing share price for INTC: (65.12, 42.64)


<h2>Step 8</h2>
<h4>15 points</h4>
Alter the trading strategy below to update the security_log table<br>
transaction_id = YYYYMMDD_symbol_random number, 100 up to 999<br>
trade_date <br>
symbol <br>
trans_type BUY OR SELL<br>
shares number of shares<br>
cost =  -1 * close_price * shares if a BUY, close_price * shares if a SELL<br>
<br>
<code>
sql = '''
SELECT symbol,trade_date,close_price - open_price as diff,close_price 
FROM security_master 
WHERE symbol = 'AMD'; '''
curr.execute(sql)
rows = curr.fetchall()
up_days = 0
down_days = 0
tot_val = 0
tot_shares = 0
share_size = 10000
for row in rows:
    symbol = row[0]
    y,m,d = str(row[1].year), ('00'+str(row[1].month))[-2:], ('00'+str(row[1].day))[-2:]
    transaction_id = f'{y}{m}{d}_{row[0]}_{str(random.randint(100,1000))}'
    trade_date = row[1]    
    '''
    if closed up for 5 consecutive days SELL 10000 shares
    sell if down for 5 consecute days '''
    if row[2] > 0:
        up_days += 1
        down_days = 0
    else:
        up_days = 0
        down_days += 1
    if up_days >= 5:
        down_days = 0
        '''SELL signal'''
        if tot_shares > 0:
            factor = 2 if tot_shares - (share_size*2) > 0 else 1
            tot_val += ((share_size*factor) * row[3])
            tot_shares -= (share_size*factor)
            # = YYYYMMDD_symbol_random number, 100 up to 999<br>
            print(f'****** SELL HERE {row[0]} {row[1]} {row[3]}  - {tot_shares} {tot_val}') 
            '''----- ADD CODE BELOW --------------'''
            
    if down_days >= 5:
        up_days = 0
        down_days = 0
        '''buy signal'''
        tot_val += (-1 * share_size * row[3])
        tot_shares += share_size
        print(f'****** BUY HERE {row[0]} {row[1]} {row[3]} - {tot_shares} {tot_val}')        
        '''----- ADD CODE BELOW --------------'''
</code>

In [437]:
import glob
import os
import random

conn = sqlite3.connect(f'stock_analysis.db')
c = conn.cursor()

c.execute("SELECT symbol,trade_date,close_price - open_price as diff,close_price FROM security_master WHERE symbol = 'AMD';")
rows = c.fetchall()
# print (rows)
up_days = 0
down_days = 0
tot_val = 0
tot_shares = 0
cost = 0
share_size = 10000
c.execute('DELETE FROM security_log')
conn.commit()
for row in rows:
    symbol = row[0]
    date_list = (row[1].split('-'))
    y,m,d = str(date_list[0]), '00'+str(date_list[1]), '00'+str(date_list[2])
    # print (y,m,d)
    transaction_id = f'{y}{m}{d}_{row[0]}_{str(random.randint(100,1000))}'
    # print (transaction_id)
    trade_date = row[1]    
    '''
    if closed up for 5 consecutive days SELL 10000 shares
    sell if down for 5 consecute days 
    '''
    # symbol,row[1] = trade_date,row[2] = close_price - open_price as diff, row[3] = close_price 
    if row[2] > 0:
        up_days += 1
        down_days = 0
    else:
        up_days = 0
        down_days += 1
    if up_days >= 5:
        down_days = 0
        '''SELL signal'''
        if tot_shares > 0:
            factor = 2 if tot_shares - (share_size*2) > 0 else 1
            cost = (share_size*factor) * row[3]
            tot_val += ((share_size*factor) * row[3])
            tot_shares -= (share_size*factor)
            print(f'****** SELL HERE {row[0]} {row[1]} {row[3]}  - {tot_shares} {tot_val}') 
            '''----- ADD CODE BELOW --------------'''
            trans_type = 'SELL' 
            pop_data = (transaction_id, trade_date, symbol, trans_type, tot_shares, round(cost,2),round(tot_val,2))
            # print (pop_data)
            c.execute('''INSERT INTO security_log (transaction_id, trade_date, symbol, trans_type, shares, cost, balance) VALUES (?,?,?,?,?,?,?)''',pop_data)
            conn.commit()

    if down_days >= 5:
        up_days = 0
        down_days = 0
        '''buy signal'''
        cost = -1 * share_size * row[3]
        tot_val += (-1 * share_size * row[3])
        # print (tot_val)
        tot_shares += share_size
        print(f'****** BUY HERE {row[0]} {row[1]} {row[3]} - {tot_shares} {tot_val}')        
        '''----- ADD CODE BELOW --------------'''
        trans_type = 'BUY'
        pop_data = (transaction_id, trade_date, symbol, trans_type, tot_shares, round(cost,2),round(tot_val,2))
        # print (pop_data)
        c.execute('''INSERT INTO security_log (transaction_id, trade_date, symbol, trans_type, shares, cost, balance) VALUES (?,?,?,?,?,?,?)''',pop_data)
        conn.commit()
conn.close()

****** BUY HERE AMD 2019-02-07 22.67 - 10000 -226700.00000000003
****** BUY HERE AMD 2019-03-28 25.059999 - 20000 -477299.99000000005
****** BUY HERE AMD 2019-06-25 28.860001 - 30000 -765900.0
****** SELL HERE AMD 2019-07-09 33.150002  - 10000 -102899.95999999996
****** SELL HERE AMD 2019-07-10 33.790001  - 0 235000.05
****** BUY HERE AMD 2019-08-06 28.860001 - 10000 -53599.96000000002
****** SELL HERE AMD 2019-11-14 38.349998  - 0 329900.01999999996
****** BUY HERE AMD 2020-05-01 49.880001 - 10000 -168899.99000000005
****** SELL HERE AMD 2020-09-30 81.989998  - 0 650999.99
****** BUY HERE AMD 2020-10-20 81.559998 - 10000 -164599.99
****** BUY HERE AMD 2020-12-22 93.160004 - 20000 -1096200.03
****** BUY HERE AMD 2021-01-19 89.449997 - 30000 -1990700.0
****** BUY HERE AMD 2021-01-28 87.519997 - 40000 -2865899.97
****** BUY HERE AMD 2021-02-22 85.370003 - 50000 -3719600.0
****** BUY HERE AMD 2021-03-08 73.959999 - 60000 -4459199.99
****** BUY HERE AMD 2021-05-03 78.550003 - 70000 -524470

<h2>Step 9</h2>
<h4>10 points</h4>
If you missed the update on step 8 above manual run the following inserts<br>
Then return the symbol,stock price,trans_type For all 2020 trades<br>

INSERT INTO security_log
        (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
        VALUES 
        ('20190207_AMD_205','2019-02-07','AMD','BUY',
        10000,-226700.0,-226700.0)

INSERT INTO security_log
        (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
        VALUES 
        ('20190328_AMD_354','2019-03-28','AMD','BUY',
        20000,-250599.99,-477299.99)
        
INSERT INTO security_log
        (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
        VALUES 
        ('20190625_AMD_336','2019-06-25','AMD','BUY',
        30000,-288600.01,-765900.0)
        
INSERT INTO security_log
            (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
            VALUES 
            ('20190709_AMD_348','2019-07-09','AMD','SELL',
            10000,331500.02,-102899.96)
            
INSERT INTO security_log
            (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
            VALUES 
            ('20190710_AMD_191','2019-07-10','AMD','SELL',
            0,337900.01,235000.05)
            
INSERT INTO security_log
        (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
        VALUES 
        ('20190806_AMD_645','2019-08-06','AMD','BUY',
        10000,-288600.01,-53599.96)
        
INSERT INTO security_log
            (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
            VALUES 
            ('20191114_AMD_174','2019-11-14','AMD','SELL',
            0,383499.98,329900.02)
            
INSERT INTO security_log
        (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
        VALUES 
        ('20200501_AMD_727','2020-05-01','AMD','BUY',
        10000,-498800.01,-168899.99)
        
INSERT INTO security_log
            (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
            VALUES 
            ('20200930_AMD_691','2020-09-30','AMD','SELL',
            0,819899.98,650999.99)
            
INSERT INTO security_log
        (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
        VALUES 
        ('20201020_AMD_135','2020-10-20','AMD','BUY',
        10000,-815599.98,-164599.99)
        
INSERT INTO security_log
        (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
        VALUES 
        ('20201222_AMD_392','2020-12-22','AMD','BUY',
        20000,-931600.04,-1096200.03)
        
INSERT INTO security_log
        (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
        VALUES 
        ('20210119_AMD_612','2021-01-19','AMD','BUY',
        30000,-894499.97,-1990700.0)
        
INSERT INTO security_log
        (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
        VALUES 
        ('20210128_AMD_454','2021-01-28','AMD','BUY',
        40000,-875199.97,-2865899.97)
        
INSERT INTO security_log
        (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
        VALUES 
        ('20210222_AMD_258','2021-02-22','AMD','BUY',
        50000,-853700.03,-3719600.0)
        
INSERT INTO security_log
        (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
        VALUES 
        ('20210308_AMD_803','2021-03-08','AMD','BUY',
        60000,-739599.99,-4459199.99)
        
INSERT INTO security_log
        (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
        VALUES 
        ('20210503_AMD_543','2021-05-03','AMD','BUY',
        70000,-785500.03,-5244700.02)

INSERT INTO security_log
            (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
            VALUES 
            ('20210528_AMD_974','2021-05-28','AMD','SELL',
            50000,800800.02,-3643099.98)
            
INSERT INTO security_log
        (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
        VALUES 
        ('20210716_AMD_346','2021-07-16','AMD','BUY',
        60000,-858899.99,-4501999.97)
        
INSERT INTO security_log
            (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
            VALUES 
            ('20210723_AMD_399','2021-07-23','AMD','SELL',
            40000,921500.02,-2658999.93)
            
INSERT INTO security_log
            (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
            VALUES 
            ('20210803_AMD_858','2021-08-03','AMD','SELL',
            20000,1125599.98,-407799.97)
            
INSERT INTO security_log
            (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
            VALUES 
            ('20210804_AMD_569','2021-08-04','AMD','SELL',
            10000,1187699.97,779900.0)
            
INSERT INTO security_log
        (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
        VALUES 
        ('20210913_AMD_796','2021-09-13','AMD','BUY',
        20000,-1048000.03,-268100.03)
        
INSERT INTO security_log
            (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
            VALUES 
            ('20210927_AMD_123','2021-09-27','AMD','SELL',
            10000,1081600.04,813500.01)

In [ ]:
import glob
import os

conn = sqlite3.connect(f'stock_analysis.db')
c = conn.cursor()

c.execute('''
INSERT INTO security_log
        (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
        VALUES 
        ('20190207_AMD_205','2019-02-07','AMD','BUY',10000,-226700.0,-226700.0),
        ('20190328_AMD_354','2019-03-28','AMD','BUY',20000,-250599.99,-477299.99),
        ('20190625_AMD_336','2019-06-25','AMD','BUY',30000,-288600.01,-765900.0),
        ('20190709_AMD_348','2019-07-09','AMD','SELL',10000,331500.02,-102899.96),
        ('20190710_AMD_191','2019-07-10','AMD','SELL',0,337900.01,235000.05),
        ('20190806_AMD_645','2019-08-06','AMD','BUY',10000,-288600.01,-53599.96),
        ('20191114_AMD_174','2019-11-14','AMD','SELL',0,383499.98,329900.02),
        ('20200501_AMD_727','2020-05-01','AMD','BUY',10000,-498800.01,-168899.99),
        ('20200930_AMD_691','2020-09-30','AMD','SELL',0,819899.98,650999.99),
        ('20201020_AMD_135','2020-10-20','AMD','BUY',10000,-815599.98,-164599.99),
        ('20201222_AMD_392','2020-12-22','AMD','BUY',20000,-931600.04,-1096200.03),
        ('20210119_AMD_612','2021-01-19','AMD','BUY',30000,-894499.97,-1990700.0),
        ('20210128_AMD_454','2021-01-28','AMD','BUY',40000,-875199.97,-2865899.97),
        ('20210222_AMD_258','2021-02-22','AMD','BUY',50000,-853700.03,-3719600.0),
        ('20210308_AMD_803','2021-03-08','AMD','BUY',60000,-739599.99,-4459199.99),
        ('20210503_AMD_543','2021-05-03','AMD','BUY',70000,-785500.03,-5244700.02),
        ('20210528_AMD_974','2021-05-28','AMD','SELL',50000,800800.02,-3643099.98),
        ('20210716_AMD_346','2021-07-16','AMD','BUY',60000,-858899.99,-4501999.97),
        ('20210723_AMD_399','2021-07-23','AMD','SELL',40000,921500.02,-2658999.93),
        ('20210803_AMD_858','2021-08-03','AMD','SELL',20000,1125599.98,-407799.97),
        ('20210804_AMD_569','2021-08-04','AMD','SELL',10000,1187699.97,779900.0),
        ('20210913_AMD_796','2021-09-13','AMD','BUY',20000,-1048000.03,-268100.03),
        ('20210927_AMD_123','2021-09-27','AMD','SELL',10000,1081600.04,813500.01)
''')      
conn.commit() 

conn.close()

In [432]:
# the symbol,stock price,trans_type For all 2020 trades

import glob
import os

conn = sqlite3.connect(f'stock_analysis.db')
c = conn.cursor()

c.execute('SELECT A.symbol, B.close_price, A.trans_type FROM security_log A LEFT JOIN security_master B ON B.trade_date = A.trade_date WHERE A.trade_date LIKE "2020%" AND A.symbol = B.symbol') 
into_data = c.fetchall()
# print (into_data)
print ('For all the trades in 2020:')
[print (f' - {i[0]}: Stock price {round(i[1],2)}, Transaction {i[2]} ') for i in into_data]
conn.close()

For all the trades in 2020:
 - AMD: Stock price 49.88, Transaction BUY 
 - AMD: Stock price 81.99, Transaction SELL 
 - AMD: Stock price 81.56, Transaction BUY 
 - AMD: Stock price 93.16, Transaction BUY 


<h2>Step 10</h2>
<h4>10 points</h4>
If you bought 100,000 shares of FB the first trading day of 2020 and sold<br>
100,000 shares on the last day of 2020. How much would you have gained or lost?

In [433]:
import glob
import os

conn = sqlite3.connect(f'stock_analysis.db')
c = conn.cursor()

c.execute('SELECT trade_date, close_price FROM security_master WHERE symbol = "FB" AND trade_date LIKE "2020%" ORDER BY trade_date ASC LIMIT 1') 
first_day = c.fetchone()
print (f'The first trading day is: {first_day[0]}, Stock price is {first_day[1]}')
c.execute('SELECT trade_date, close_price FROM security_master WHERE symbol = "FB" AND trade_date LIKE "2020%" ORDER BY trade_date DESC LIMIT 1') 
last_day = c.fetchone()
print (f'The last trading day is: {last_day[0]}, Stock price is {last_day[1]}')

winnings = round((last_day[1]-first_day[1])*100000,2)
print (f' The winnings for 100,000 shares is: {winnings}')

conn.close()


The first trading day is: 2020-01-02, Stock price is 209.779999
The last trading day is: 2020-12-31, Stock price is 273.160004
 The winnings for 100,000 shares is: 6338000.5


<h2>Bonus Question</h2>
<h4>25 points</h4>
Write a program that buys 10000 shares of INTC every time the volume<br>
is greater than 10% for the average daily volume for 10 days and sell 10000 shares <br>
when the average volume for the past 5 days.  The initial buy should be when the shares <br>
have had 5 consecutive positive trading days


Bonus Question
25 points
1 - The initial buy should be when the shares of INTC
have had 5 consecutive positive trading days. Buy 10000 shares
2 - Buy 10000 shares of INTC every time the volume
is greater than 10% for the average daily volume for 10 days
3 - sell 10000 shares when the average volume for the past 5 days is
less than 10% of the trailing volume
What is the profit or loss . All shares have to be traded and no shorts allowed

In [445]:
import glob
import os

conn = sqlite3.connect(f'stock_analysis.db')
c = conn.cursor()

c.execute("SELECT symbol,trade_date,close_price FROM security_master WHERE symbol = 'AMD';")
# c.execute("SELECT symbol,trade_date,close_price - open_price as diff,close_price FROM security_master WHERE symbol = 'AMD';")
rows = c.fetchall()
# print(rows)
up_days = 0
down_days = 0
tot_val = 0
tot_shares = 0
cost = 0
count = 1 
share_size = 10000
conn.commit()
close_price_list = [row[2] for row in rows]
# print (close_price_list, len(close_price_list)) 
for row in rows:
    symbol = row[0]
    date_list = (row[1].split('-'))
    y,m,d = str(date_list[0]), '00'+str(date_list[1]), '00'+str(date_list[2])
    transaction_id = f'{y}{m}{d}_{row[0]}_{str(random.randint(100,1000))}'
    # print (transaction_id)
    trade_date = row[1]    
    '''
    if closed up for 5 consecutive days SELL 10000 shares
    sell if down for 5 consecute days 
    '''

    try:
        for i in range(len(close_price_list)):
            print (close_price_list[1], close_price_list[0]) 
    except:
    # symbol,/row[1] = trade_date,/row[2] = vol,/ row[3] = close_price 
    # if row[2] > 0: #10% for the average daily volume
    #     up_days += 1
    #     down_days = 0
    # else:
    #     up_days = 0
    #     down_days += 1
    # if up_days >= 5 and count == 1:
    #     count +=1
    #     up_days = 0
    #     down_days = 0
    #     '''buy signal'''
    #     cost = -1 * share_size * row[3]
    #     tot_val += (-1 * share_size * row[3]) # Valance
    #     # print (tot_val)
    #     tot_shares += share_size
    #     print(f'****** BUY HERE {row[0]} {row[1]} {row[3]} - {tot_shares} {tot_val}')        
    #     '''----- ADD CODE BELOW --------------'''
    #     trans_type = 'BUY'
    #     pop_data = (transaction_id, trade_date, symbol, trans_type, tot_shares, round(cost,2),round(tot_val,2))
    #     print (pop_data)
    #     c.execute('''INSERT INTO security_log (transaction_id, trade_date, symbol, trans_type, shares, cost, balance) VALUES (?,?,?,?,?,?,?)''',pop_data)
    #     conn.commit()
    

20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.66 21.49
20.6

In [415]:
# 2 - Buy 10000 shares of INTC every time the volume is greater than 10% for the average daily volume for 10 days

import glob
import os

conn = sqlite3.connect(f'stock_analysis.db')
c = conn.cursor()
c.execute('SELECT volume, trade_date FROM security_master WHERE symbol = "INTC" ORDER BY trade_date')
rows = c.fetchall()
vol_10 = [rows[i][0] for i in range (10)]
avg_vol = sum(vol_10)/len(vol_10)
print (avg_vol + avg_vol*.1)





# if up_days >= 5:
#         down_days = 0
#         '''SELL signal'''
#         if tot_shares > 0:
#             factor = 2 if tot_shares - (share_size*2) > 0 else 1
#             cost = (share_size*factor) * row[3]
#             tot_val += ((share_size*factor) * row[3])
#             tot_shares -= (share_size*factor)
#             print(f'****** SELL HERE {row[0]} {row[1]} {row[3]}  - {tot_shares} {tot_val}') 
#             '''----- ADD CODE BELOW --------------'''
#             trans_type = 'SELL' 
#             pop_data = (transaction_id, trade_date, symbol, trans_type, tot_shares, round(cost,2),round(tot_val,2))
#             # print (pop_data)
#             c.execute('''INSERT INTO security_log (transaction_id, trade_date, symbol, trans_type, shares, cost, balance) VALUES (?,?,?,?,?,?,?)''',pop_data)
#             conn.commit()

#     if down_days >= 5:
#         up_days = 0
#         down_days = 0
#         '''buy signal'''
#         cost = -1 * share_size * row[3]
#         tot_val += (-1 * share_size * row[3])
#         # print (tot_val)
#         tot_shares += share_size
#         print(f'****** BUY HERE {row[0]} {row[1]} {row[3]} - {tot_shares} {tot_val}')        
#         '''----- ADD CODE BELOW --------------'''
#         trans_type = 'BUY'
#         pop_data = (transaction_id, trade_date, symbol, trans_type, tot_shares, round(cost,2),round(tot_val,2))
#         # print (pop_data)
#         c.execute('''INSERT INTO security_log (transaction_id, trade_date, symbol, trans_type, shares, cost, balance) VALUES (?,?,?,?,?,?,?)''',pop_data)
#         conn.commit()
# conn.close()










# c.execute('SELECT AVG(volume) FROM security_master WHERE symbol = "INTC" LIMIT 10')
# rows = c.fetchall()
# print (rows)
# avg_vol = rows[0] + (rows[0]*0.1)

26968634.0


In [ ]:
import glob
import os

conn = sqlite3.connect(f'stock_analysis.db')
c = conn.cursor()
c.execute('SELECT AVG(volume) FROM security_master WHERE symbol = "INTC"')
rows = c.fetchone()
avg_vol = rows[0] + (rows[0]*0.1)

c.execute("SELECT symbol,trade_date,close_price - open_price as diff,close_price FROM security_master WHERE symbol = 'AMD';")
rows = c.fetchall()
# print (rows)
up_days = 0
down_days = 0
tot_val = 0
tot_shares = 0
cost = 0
share_size = 10000
c.execute('DELETE FROM security_log')
conn.commit()
for row in rows:
    symbol = row[0]
    date_list = (row[1].split('-'))
    y,m,d = str(date_list[0]), '00'+str(date_list[1]), '00'+str(date_list[2])
    # print (y,m,d)
    transaction_id = f'{y}{m}{d}_{row[0]}_{str(random.randint(100,1000))}'
    # print (transaction_id)
    trade_date = row[1]    
    '''
    if closed up for 5 consecutive days SELL 10000 shares
    sell if down for 5 consecute days 
    '''
    # symbol,/row[1] = trade_date,/row[2] = vol,/ row[3] = close_price 
    if row[2] > avg_vol: #10% for the average daily volume
        up_days += 1
        down_days = 0
    else:
        up_days = 0
        down_days += 1
    if up_days >= 10: 
        down_days = 0
        '''SELL signal'''
        if tot_shares > 0:
            factor = 2 if tot_shares - (share_size*2) > 0 else 1
            cost = (share_size*factor) * row[3]
            tot_val += ((share_size*factor) * row[3])
            tot_shares -= (share_size*factor)
            print(f'****** SELL HERE {row[0]} {row[1]} {row[3]}  - {tot_shares} {tot_val}') 
            '''----- ADD CODE BELOW --------------'''
            trans_type = 'SELL' 
            pop_data = (transaction_id, trade_date, symbol, trans_type, tot_shares, round(cost,2),round(tot_val,2))
            # print (pop_data)
            c.execute('''INSERT INTO security_log (transaction_id, trade_date, symbol, trans_type, shares, cost, balance) VALUES (?,?,?,?,?,?,?)''',pop_data)
            conn.commit()

    if down_days >= 5:
        up_days = 0
        down_days = 0
        '''buy signal'''
        cost = -1 * share_size * row[3]
        tot_val += (-1 * share_size * row[3])
        # print (tot_val)
        tot_shares += share_size
        print(f'****** BUY HERE {row[0]} {row[1]} {row[3]} - {tot_shares} {tot_val}')        
        '''----- ADD CODE BELOW --------------'''
        trans_type = 'BUY'
        pop_data = (transaction_id, trade_date, symbol, trans_type, tot_shares, round(cost,2),round(tot_val,2))
        # print (pop_data)
        c.execute('''INSERT INTO security_log (transaction_id, trade_date, symbol, trans_type, shares, cost, balance) VALUES (?,?,?,?,?,?,?)''',pop_data)
        conn.commit()
conn.close()

****** BUY HERE AMD 2018-11-21 18.73 - 10000 -187300.0
****** BUY HERE AMD 2018-11-29 21.43 - 20000 -401600.0
****** BUY HERE AMD 2018-12-07 19.459999 - 30000 -596199.99
****** BUY HERE AMD 2018-12-14 19.9 - 40000 -795199.99
****** BUY HERE AMD 2018-12-21 16.93 - 50000 -964499.99
****** BUY HERE AMD 2018-12-31 18.459999 - 60000 -1149099.98
****** BUY HERE AMD 2019-01-08 20.75 - 70000 -1356599.98
****** BUY HERE AMD 2019-01-15 20.379999 - 80000 -1560399.97
****** BUY HERE AMD 2019-01-23 19.799999 - 90000 -1758399.96
****** BUY HERE AMD 2019-01-30 23.09 - 100000 -1989299.96
****** BUY HERE AMD 2019-02-06 23.26 - 110000 -2221899.96
****** BUY HERE AMD 2019-02-13 22.85 - 120000 -2450399.96
****** BUY HERE AMD 2019-02-21 23.92 - 130000 -2689599.96
****** BUY HERE AMD 2019-02-28 23.530001 - 140000 -2924899.9699999997
****** BUY HERE AMD 2019-03-07 22.08 - 150000 -3145699.9699999997
****** BUY HERE AMD 2019-03-14 22.82 - 160000 -3373899.9699999997
****** BUY HERE AMD 2019-03-21 27.889999 - 17

In [ ]:
import glob
import os

conn = sqlite3.connect(f'stock_analysis.db')
c = conn.cursor()
c.execute('SELECT AVG(volume) FROM security_master WHERE symbol = "INTC"')
rows = c.fetchone()
avg_vol = rows[0] + (rows[0]*0.1)

c.execute("SELECT symbol,trade_date,close_price - open_price as diff,close_price FROM security_master WHERE symbol = 'AMD';")
rows = c.fetchall()
# print (rows)
up_days = 0
down_days = 0
tot_val = 0
tot_shares = 0
cost = 0
share_size = 10000
c.execute('DELETE FROM security_log')
conn.commit()
for row in rows:
    symbol = row[0]
    date_list = (row[1].split('-'))
    y,m,d = str(date_list[0]), '00'+str(date_list[1]), '00'+str(date_list[2])
    # print (y,m,d)
    transaction_id = f'{y}{m}{d}_{row[0]}_{str(random.randint(100,1000))}'
    # print (transaction_id)
    trade_date = row[1]    
    '''
    if closed up for 5 consecutive days SELL 10000 shares
    sell if down for 5 consecute days 
    '''
    # symbol,/row[1] = trade_date,/row[2] = vol,/ row[3] = close_price 
    if row[2] > avg_vol: #10% for the average daily volume
        up_days += 1
        down_days = 0
    else:
        up_days = 0
        down_days += 1
    if up_days >= 10: 
        down_days = 0
        '''SELL signal'''
        if tot_shares > 0:
            factor = 2 if tot_shares - (share_size*2) > 0 else 1
            cost = (share_size*factor) * row[3]
            tot_val += ((share_size*factor) * row[3])
            tot_shares -= (share_size*factor)
            print(f'****** SELL HERE {row[0]} {row[1]} {row[3]}  - {tot_shares} {tot_val}') 
            '''----- ADD CODE BELOW --------------'''
            trans_type = 'SELL' 
            pop_data = (transaction_id, trade_date, symbol, trans_type, tot_shares, round(cost,2),round(tot_val,2))
            # print (pop_data)
            c.execute('''INSERT INTO security_log (transaction_id, trade_date, symbol, trans_type, shares, cost, balance) VALUES (?,?,?,?,?,?,?)''',pop_data)
            conn.commit()

    if down_days >= 5:
        up_days = 0
        down_days = 0
        '''buy signal'''
        cost = -1 * share_size * row[3]
        tot_val += (-1 * share_size * row[3])
        # print (tot_val)
        tot_shares += share_size
        print(f'****** BUY HERE {row[0]} {row[1]} {row[3]} - {tot_shares} {tot_val}')        
        '''----- ADD CODE BELOW --------------'''
        trans_type = 'BUY'
        pop_data = (transaction_id, trade_date, symbol, trans_type, tot_shares, round(cost,2),round(tot_val,2))
        # print (pop_data)
        c.execute('''INSERT INTO security_log (transaction_id, trade_date, symbol, trans_type, shares, cost, balance) VALUES (?,?,?,?,?,?,?)''',pop_data)
        conn.commit()
conn.close()

****** BUY HERE AMD 2018-11-21 18.73 - 10000 -187300.0
****** BUY HERE AMD 2018-11-29 21.43 - 20000 -401600.0
****** BUY HERE AMD 2018-12-07 19.459999 - 30000 -596199.99
****** BUY HERE AMD 2018-12-14 19.9 - 40000 -795199.99
****** BUY HERE AMD 2018-12-21 16.93 - 50000 -964499.99
****** BUY HERE AMD 2018-12-31 18.459999 - 60000 -1149099.98
****** BUY HERE AMD 2019-01-08 20.75 - 70000 -1356599.98
****** BUY HERE AMD 2019-01-15 20.379999 - 80000 -1560399.97
****** BUY HERE AMD 2019-01-23 19.799999 - 90000 -1758399.96
****** BUY HERE AMD 2019-01-30 23.09 - 100000 -1989299.96
****** BUY HERE AMD 2019-02-06 23.26 - 110000 -2221899.96
****** BUY HERE AMD 2019-02-13 22.85 - 120000 -2450399.96
****** BUY HERE AMD 2019-02-21 23.92 - 130000 -2689599.96
****** BUY HERE AMD 2019-02-28 23.530001 - 140000 -2924899.9699999997
****** BUY HERE AMD 2019-03-07 22.08 - 150000 -3145699.9699999997
****** BUY HERE AMD 2019-03-14 22.82 - 160000 -3373899.9699999997
****** BUY HERE AMD 2019-03-21 27.889999 - 17

In [ ]:
import glob
import os

conn = sqlite3.connect(f'stock_analysis.db')
c = conn.cursor()
# c.execute('SELECT round((AVG(volume)+AVG(volume)*0.1),2) FROM security_master WHERE symbol = "INTC"')
c.execute('SELECT AVG(volume) FROM security_master WHERE symbol = "INTC"')
rows = c.fetchone()
row = rows[0] + (rows[0]*0.1)
# print (row)
c.execute(f'SELECT trade_date, volume FROM security_master WHERE symbol = "INTC" AND volume > {row} ORDER BY trade_date ASC LIMIT 10')
vol_rows = c.fetchall()
[print (vol_row) for vol_row in vol_rows]

conn.close()
